# Data curation
* Modelling over Vanuatu Espiritu Santo
* UTM Zone 58S (162E - 168E) **_EPSG:32758_**
_However the tile_index file is coded for S59 and their is no CRS information for the LAZ files so may be for the wrong CRS. Try UTM 58S to stat._

## Data avaliable
* LiDAR survey (populated coastal areas)
* FABDEM V1-1 - https://data.bris.ac.uk/data/dataset/s5hqmjcdj8yo2ibzi9b4ew3sn
* FABDEM V1-0 - https://data.bris.ac.uk/data/dataset/25wfy0f9ukoge2gs7a5mqpq2j7

## Vanuata Espiritu Santo
* Located 166E-168E, 14S to 16S -> FABDEM V1-2 S20E160-S10E170

# Tasks to do
* Stictch Coarse DEM together (limitation of GeoFabrics)
  a. Combine the tiles over Espiritu Santo
* Copy data to NeSI - WSL scp
  a. Coarse DEM - FABDEM S20E160-S10E170
  b. LiDAR Tile Index - \\niwa.local\projects\hamilton\GNS23201\RawData\LiDAR\EspirituSanto_FullRawLidar via my computer
  c. LiDAR tiles - \\niwa.local\projects\hamilton\GNS23201\RawData\LiDAR\EspirituSanto_FullRawLidar
* Create extents
  a. A small extents at edge of LiDAR & Coarse DEM for testing
  b. The bottom 1/3 of the island

In [6]:
import pathlib
import rioxarray, rioxarray.merge

local_path = pathlib.Path(r"C:\Local\data\GNS23201\CoarseDEMs")
fab_path = local_path / "S20E160-S10E170_FABDEM_V1-2"

# Espiritu Santo
lat_lons = [[15, 166], [15, 167], [16, 166], [16, 167]]
fabs = []
for lat_lon in lat_lons:
    fabs.append(rioxarray.open_rasterio(raw_path / f"S{lat_lon[0]}E{lat_lon[1]}_FABDEM_V1-2.tif", masked=True))
fab = rioxarray.merge.merge_arrays(fabs).rio.reproject(32758)
fab.rio.write_nodata(fab.rio.nodata).to_netcdf(local_path / "espiritu_FABDEM_EPSG32758.nc")

In [11]:
import shapely
import geopandas

local_path = pathlib.Path(r"C:\Local\data\GNS23201\extents")

x0=699229; x1=700764
y0=8276469; y1=8274187
small=geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x0,y0],[x1,y0],[x1,y1],[x0,y1]])], crs=32758)
if not (local_path / "small.geojson").exists():
    small.to_file(local_path / "small.geojson")

x0=748103; x1=677208
y0=8307897; y1=8265823
big=geopandas.GeoDataFrame(geometry=[shapely.geometry.Polygon([[x0,y0],[x1,y0],[x1,y1],[x0,y1]])], crs=32758)
if not (local_path / "big.geojson").exists():
    big.to_file(local_path / "big.geojson")

In [17]:
dx = round((x0-x1)/10); dy = round((y0-y1)/3)

In [22]:
all_tiles = []
for i, xi in enumerate(range(x1, x0, dx)):
    for j, yi in enumerate(range(y1, y0, dy)):
        tile = shapely.geometry.Polygon([[xi,yi],[xi+dx,yi],[xi+dx,yi+dy],[xi,yi+dy]])
        all_tiles.append(tile)
        tile = geopandas.GeoDataFrame(geometry=[tile], crs=32758)
        tile.to_file(local_path / f"tile_{i}{j}.geojson")
geopandas.GeoDataFrame(geometry=all_tiles, crs=32758).to_file(local_path / f"all_tiles.geojson")